# Case Study Data Preparation

msFeaST runs with three linked data structures in the form of:

1. A quantification table with sample specific feature intensities. The column names are assumed to be sample identifiers, with a suffix (e.g. "Peak Area ", beware of whitespace sensitivity)
2. a metadata table with sample identifier to treatment identifier mapping (column names are sample_id, and treatment_id)
3. a mgf file with spectral data for features

Feature identifiers, sample identifiers, and treatment identifiers are assumed matching across files. 
In this jupyter notebook we showcase data extraction for the illustrative examples using python's pandas package. 
While data set-up for msFeaSt will be quite straightforward using exisiting gnps-fbmn exports (from gnps or msmine3), some additional data modification may need to happen to make sure statistical data is in the correct layout and appropriate contrasts are selected.

# Import Python Dependencies
This code chunk loads python package dependencies for data processing. All packages are installed by default when following the msFeaST installation guide.

In [3]:
import matchms
import pandas as pd
import numpy as np
import plotly
import copy
import os

# Extract & Re-format data
GNPS-FBMN network data contains numerous entries not requires by the msFeaST workflow. In the following processing steps, the input data is processed to contain only relevant data as expected by the msFeaST pipeline. We delineate between general steps and mushroom data specific steps to allow users to customize these steps to their own data. Unfortunately, given the metadata specific and thus unique setting of each dataset, complete automatization of this process is not possible. Users will have to make sure that they have right data available to get to the expected pipeline input.

## Loading raw metadata

Note that the raw data file is placed inside the data/mushroom_data_gnps_export folder and named metadata.tsv, in tab separated format (.tsv). The mushroom dataset contains numerous samples not of direct relevance to the statistical analyses we're performing in msFeaST. The relevant data subsets must hence be extracted for the automatic analysis pipeline of msFeaST to make use of the correct data in following steps.

In [13]:
raw_metadata_filepath = os.path.join("data", "mushroom_data_gnps_export", "metadata.tsv")
raw_statistical_metadata = pd.read_table(raw_metadata_filepath)
print("Data dimensions (number of rows, number of columns):", raw_statistical_metadata.shape)
raw_statistical_metadata.head()

Data dimensions (number of rows, number of columns): (54, 12)


,filename,SampleType,SampleType1,ATTRIBUTE_ Percent of OMSW in MS,Species,ATTRIBUTE_ Taxonomy,NCBITaxonomy,Sample Collection,Sample Extract,MassSpectrometer,IonizationSourceAndPolarity,ChromatographyAndPhase
0,MS0_NEW_POS.mzXML,BLANK_MS,Mushroom Substrate,0,Fungi,MS,91752Hericium erinaceus,Dry Solid Material,Methanol100%,Q Exactive Plus|MS:1002661,electrospray ionization (Positive),reverse phase (C18)
1,MS0_OLD_POS.mzXML,BLANK_MS,Mushroom Substrate,0,Fungi,MS,91752Hericium erinaceus,Dry Solid Material,Methanol100%,Q Exactive Plus|MS:1002662,electrospray ionization (Positive),reverse phase (C18)
2,MS33_NEW_POS.mzXML,BLANK_MS,Mushroom Substrate,33,Fungi,MS,91752Hericium erinaceus,Dry Solid Material,Methanol100%,Q Exactive Plus|MS:1002663,electrospray ionization (Positive),reverse phase (C18)
3,MS33_OLD_POS.mzXML,BLANK_MS,Mushroom Substrate,33,Fungi,MS,91752Hericium erinaceus,Dry Solid Material,Methanol100%,Q Exactive Plus|MS:1002664,electrospray ionization (Positive),reverse phase (C18)
4,MS60_NEW_POS.mzXML,BLANK_MS,Mushroom Substrate,60,Fungi,MS,91752Hericium erinaceus,Dry Solid Material,Methanol100%,Q Exactive Plus|MS:1002665,electrospray ionization (Positive),reverse phase (C18)


## Loading Raw Quantification Table

Note that the raw data file is placed inside the data/mushroom_data_gnps_export folder and named quantification_table.csv, in comma separated format (.csv). Each row contains a 'row ID' column identifying a feature. Each feature has associated precursor m/z value within the 'row m/z' column, as well as retention time in 'row retention time'. Note that units for retention time are not given. The data required by msFeaST are feature specific intensity profiles across samples indicated by columns with the following name construct >>sample identifier + ' Peak area'<<, e.g., 'E37_pos.mzXML Peak area'.

Note that the raw data import into pandas leads to many columns with NaN (not available number) entries and somewhat complex column naming conventions that prevent direct matching to sample identifiers because of the 'Peak area' suffix. These data features are dealt with in the processing code below.

In [15]:
raw_quant_table_filepath = os.path.join("data", "mushroom_data_gnps_export", "quantification_table.csv")
quantification_table = pd.read_csv(raw_quant_table_filepath)
print("Data dimensions (number of rows, number of columns):", quantification_table.shape)
quantification_table.head()

Data dimensions (number of rows, number of columns): (2984, 68)


,row ID,row m/z,row retention time,row ion mobility,row ion mobility unit,row CCS,correlation group ID,annotation network number,best ion,auto MS2 verify,...,E37_pos.mzXML Peak area,E38_pos.mzXML Peak area,E39_pos.mzXML Peak area,E36_pos.mzXML Peak area,E43_pos.mzXML Peak area,E40_pos.mzXML Peak area,E41_pos.mzXML Peak area,E44_pos.mzXML Peak area,E42_pos.mzXML Peak area,Unnamed: 67
0,555,69.034280,1.209123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7182873.000,3206877.000,6456761.500,5544007.500,6295892.000,1.058997e+07,7853519.000,7683877.000,9745292.000,NaN
1,994,70.065890,1.216007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,536219.100,1073616.500,370348.060,682284.700,290696.380,3.698430e+05,387468.440,333006.600,294155.340,NaN
2,15743,71.086306,17.530378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37769.434,22959.324,22191.406,41042.824,17818.375,1.655038e+04,25448.713,33429.113,57842.637,NaN
3,2563,79.054930,5.326331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,499695.280,514116.400,707964.060,479117.060,552020.900,9.169636e+05,717532.200,790312.940,951237.250,NaN
4,8783,83.049808,13.057878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,88554.445,58410.418,28739.414,25085.014,23766.312,7.357520e+04,27015.748,47035.324,85310.110,NaN


## Loading raw spectral data
Similar to the other raw data, the raw spectral data from the gnps export may contain compatibility artefacts. For instance, some ms/ms features may have empty or very low amounts of spectral data. Spectral data are loaded and processed using matchms within the msFeaST workflow. While the initial number of features in the spectral data file is large, post-processing drastically reduces this number, especially via the minimum fragment number required. Setting the minimum number of fragment to some lower-bound is adviseable since a lack of spectral data information will prevent meaningful spectral similarity scoring and thus only introduce complexity and noise into the workflow.


In [39]:
raw_spectra_filepath = os.path.join("data", "mushroom_data_gnps_export", "spectra.mgf")
raw_spectra = list(matchms.importing.load_from_mgf(raw_spectra_filepath))

In [50]:
from msfeast.preprocessing import apply_default_spectral_processing
spectra = apply_default_spectral_processing(raw_spectra)

Number of spectral features provided:  18562
Number of spectral features which passed pre-processing:  2910


## Extracting, Transforming, and loading the data for msFeaST compatibility

The quantification table, metadata table, and spectral data loaded form the basis of msFeaST. However, they contain redundant information pieces still. Not all columns in the metadata table are relevant, nor are all rows. Not all samples in the quantification table are used. Depending on processing and subsetting, we may end with spectra which do not contain intensity information in any of the samples intended for analysis. There is hence a need for loading the data and processing it to remove irrelevant or incompatible information pieces.


step 1: DONE
--> define sample space via metadata, ensure correct pandas states (str etc)
--> Make sure sample id and treatment id conform to string and uniqueness, add sanity check the treatment_ids; make sure pandas index isn't an issue
--> allow re-order to implicitly force reference category (first)


step 2:
--> convert quantification table to correct format. feature_id vs sample_id df, make sure pandas index isn't an issue

step 3:
--> extract table with feature_id, precursor mz, and rt columns from quantification table

step 4: 
--> extract feature_id list from spectra (assume processed using apply_defualt_spectral_processing() <=> feature_id should be in there) 

step 5:
--> from processed quant table, subset to features with data (not all 0), get feature_id exclusion list to apply to list of spectra

--> from align sample_id ordering across metadata and quantification table

step 6: 
--> subset spectra to only contain the ones also found in quant table, check feature_id sets equal

step 7 [optional] : provide total ion current or sample-centric normalization. Divide all intensities by the sum of intensities in a sample. Comes with implicit assumptions if used quantitatively.
--> Recommend as much data cleaning as possible, blank removal, batch effect correction etcs, in pre-processing tools such as MZMine3. 
--> apply sample wise total ion current scaling to deal with differential intensity effects, recommend any data to be bath effect corrected to begin with (mzmine). 



step 8:
--> export tables / expose of msfeast loading

we have the metadata file now in correct form
we have the spectra subsetted two ways (spectral filtering, quantitative filtering)
we have the quantification table processed, and subsetted to only used samples, and only used features, which are also available in the samples

# Step 1 - Extract statistical treatment data

In [203]:
%load_ext autoreload
%autoreload 2
import msfeast
from msfeast.preprocessing import extract_treatment_table

treatment_table = extract_treatment_table(
  metadata_table=raw_statistical_metadata, 
  treatment_column_name="ATTRIBUTE_ Taxonomy", 
  treatment_identifiers=['FB_Hericium', 'FB_Pleurotus'], 
  reference_category="FB_Hericium"
)

treatment_table.reset_index(drop = True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,sample_id,treatment
0,P1_pos.mzXML,FB_Hericium
1,P2_pos.mzXML,FB_Hericium
2,P3_pos.mzXML,FB_Hericium
3,P4_pos.mzXML,FB_Hericium
4,P5_pos.mzXML,FB_Hericium
5,P6_pos.mzXML,FB_Hericium
6,P7_pos.mzXML,FB_Hericium
7,P8_pos.mzXML,FB_Hericium
8,P9_pos.mzXML,FB_Hericium
9,P10_pos.mzXML,FB_Hericium


### PROCESS THE QUANTIFICAITON TABLE

In [204]:
# Step 6 - Subset spectra to features with quantification information.
from msfeast.preprocessing import subset_spectra_to_exclude, subset_spectra_to_include



In [218]:
# reformat quantification table to expectations

def restructure_quantification_table(
    quantification_table : pd.DataFrame, 
    feature_id_column_name : str = "row ID", 
    sample_id_suffix : str = "Peak area") -> pd.DataFrame:
  """ 
  Parses quantification table into expected format for msFeaST. 

  Parameters
    quantification_table : pd.DataFrame, 
    feature_id_column_name : str = "row ID", 
    sample_id_suffix : str = "Peak area")

  Returns
    pd.DataFrame with sample_id and column and one column per feature_id.
  """
  ...
  quant_table = copy.deepcopy(quantification_table)
  # Extract feature id columns and any sample id columns via sample_id suffic 
  quant_table = quant_table.filter(
    regex=f"{feature_id_column_name}|{sample_id_suffix}", axis=1
  )
  quant_table = quant_table.rename(columns = {'row ID':'feature_id'})
  quant_table = quant_table.melt(id_vars="feature_id", var_name="sample_id").reset_index(drop=True)
  quant_table["feature_id"] = quant_table["feature_id"].astype(dtype="string")
  quant_table["sample_id"] = quant_table["sample_id"].astype(dtype="string")
  quant_table["sample_id"] = quant_table["sample_id"].str.replace(pat=" Peak area", repl="")
  # Pivot creates a hierarchical index, where the the columns are named feature_id, and the new index is added
  # to as a secondary column layer when resetting the index without dropping the column. This leads to the impression
  # (visually when printing) that the index is called feature_id. To avoid this, remove the name for the column index
  quant_table = pd.pivot(quant_table, columns="feature_id", index = "sample_id", values="value").reset_index()
  quant_table.columns.name = '' 
  return quant_table

quant_table = restructure_quantification_table(quantification_table)

def normalize_via_total_ion_current(quantification_table):
  """ 
  Apply total ion current normalization to quantification table

  For each sample (row) compute sum and divide each entry by this sum.
  """
  quant_table = copy.deepcopy(quantification_table)
  # TIC code adapted from 
  # https://github.com/Functional-Metabolomics-Lab/FBMN-STATS/blob/main/Python/Stats_Untargeted_Metabolomics_python.ipynb
  numeric_data = quant_table.drop("sample_id", axis = 1)
  numeric_data = numeric_data.apply(lambda x: x / np.sum(x), axis=1) # numeric_data.sum(axis= 1) --> all 1
  quant_table = pd.concat([quant_table["sample_id"], numeric_data], axis=1)
  return quant_table

quant_table = normalize_via_total_ion_current(quant_table)

# subset quant_table to sample identifiers from treatent table

def subset_quantification_table_to_samples(quantification_table :pd.DataFrame, sample_id_list : List[str]):
  """ Subsets the quantification table to the sample identifiers provided. """
  quant_table = copy.deepcopy(quantification_table)
  quant_table = quant_table.query("sample_id in @sample_id_list")
  return quant_table
# get invalid quantification table entries (no data in sample subset)

sample_id_list = treatment_table["sample_id"].to_list()
sample_id_list
quant_table = subset_quantification_table_to_samples(quant_table, sample_id_list) # <----------------------------------- requires the sample_id list or treatment table for its derivation

In [219]:
quant_table

,sample_id,10001,10010,10012,10013,10015,10023,10026,10041,10043,...,996,9960,9963,9965,9972,9976,9986,9992,9993,9994
0,E10_pos.mzXML,0.000018,0.000000,0.000008,0.000000,0.000000e+00,0.000000e+00,0.000098,0.000000e+00,0.000000,...,1.195877e-05,0.000000,0.000223,0.000000e+00,0.000000,0.000000e+00,0.000014,0.000000e+00,0.000024,1.459552e-06
1,E11_pos.mzXML,0.000026,0.000000,0.000013,0.000000,0.000000e+00,0.000000e+00,0.000141,0.000000e+00,0.000000,...,1.833984e-05,0.000000,0.000224,0.000000e+00,0.000000,5.080331e-07,0.000015,0.000000e+00,0.000027,3.510465e-06
2,E12_pos.mzXML,0.000022,0.000000,0.000010,0.000000,0.000000e+00,0.000000e+00,0.000128,0.000000e+00,0.000000,...,1.356324e-05,0.000000,0.000131,0.000000e+00,0.000000,0.000000e+00,0.000013,0.000000e+00,0.000031,2.934514e-06
3,E1_pos.mzXML,0.000059,0.000000,0.000083,0.000004,0.000000e+00,4.438126e-07,0.000342,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000200,1.283172e-06,0.000000,3.100063e-06,0.000012,0.000000e+00,0.000210,2.324214e-05
4,E2_pos.mzXML,0.000010,0.000000,0.000066,0.000004,0.000000e+00,0.000000e+00,0.000053,9.598870e-07,0.000000,...,0.000000e+00,0.000000,0.000135,9.925285e-07,0.000000,3.183238e-06,0.000011,0.000000e+00,0.000152,1.090729e-05
12,E3_pos.mzXML,0.000022,0.000001,0.000136,0.000006,7.651379e-07,0.000000e+00,0.000116,0.000000e+00,0.000000,...,7.385226e-07,0.000000,0.000212,0.000000e+00,0.000000,4.523933e-07,0.000013,0.000000e+00,0.000101,7.958650e-07
18,E4_pos.mzXML,0.000017,0.000000,0.000090,0.000004,0.000000e+00,0.000000e+00,0.000067,0.000000e+00,0.000000,...,2.466807e-06,0.000000,0.000162,0.000000e+00,0.000000,1.055405e-06,0.000013,0.000000e+00,0.000083,4.703420e-06
19,E5_pos.mzXML,0.000018,0.000000,0.000053,0.000004,0.000000e+00,0.000000e+00,0.000116,0.000000e+00,0.000000,...,1.861800e-06,0.000000,0.000150,0.000000e+00,0.000000,6.679414e-07,0.000016,5.667687e-07,0.000050,2.128412e-06
20,E6_pos.mzXML,0.000009,0.000000,0.000035,0.000001,0.000000e+00,0.000000e+00,0.000049,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000116,0.000000e+00,0.000000,5.002292e-07,0.000010,0.000000e+00,0.000146,6.998005e-06
21,E7_pos.mzXML,0.000008,0.000000,0.000014,0.000000,0.000000e+00,0.000000e+00,0.000039,0.000000e+00,0.000000,...,0.000000e+00,0.000000,0.000185,0.000000e+00,0.000000,0.000000e+00,0.000015,0.000000e+00,0.000069,1.572612e-06


In [220]:
def generate_exclusion_list (quantification_table : pd.DataFrame):
  zero_columns = quantification_table.columns[(quantification_table == 0).all()]
  if zero_columns.empty:
      print("No columns have only zero entries.")
  else:
      print(f"Number of columns with only zero entries: {len(zero_columns)}")
  all_zero_features = list(zero_columns)
  return all_zero_features
  

from msfeast.preprocessing import subset_spectra_to_include
from typing import List
def align_feature_subsets(quantification_table : pd.DataFrame, spectra : List[matchms.Spectrum]):
  # Assumes:
  # --> treatment table is already subset, and only relevant samples are in quantification table
  # --> spectra were already subset and contain only suitable spectra (no further removal necessary)
  all_features = [spec.get("feature_id") for spec in spectra]
  exclusion_list = generate_exclusion_list(quant_table) # this is only the case after subsetting the samples sets.
  print(exclusion_list)
  overlapping_features = list(set(all_features).difference(set(exclusion_list)))
  subset_spectra = subset_spectra_to_include(spectra, overlapping_features)
  subset_quant_table = quantification_table[overlapping_features]
  return subset_quant_table, subset_spectra

# FIX: SOMEHWERE THE SAMPLE ID IS GETTING LOST BEFORE SUBSETTING.

In [221]:
qt, sp = align_feature_subsets(quant_table, spectra)

Number of columns with only zero entries: 222
['10229', '10294', '10423', '10514', '10543', '10616', '10642', '10687', '10702', '10712', '10756', '10766', '10878', '10897', '10995', '11017', '11024', '11121', '11162', '11290', '11451', '11644', '11911', '12081', '12082', '12280', '12303', '12338', '12602', '12668', '12670', '12676', '12738', '12764', '12785', '12797', '12826', '12894', '12935', '13144', '13253', '13272', '13448', '13537', '13669', '13801', '13900', '14077', '14082', '14113', '14177', '14304', '14323', '14450', '14707', '14763', '14818', '14912', '15302', '15362', '15576', '15664', '15827', '15903', '15926', '15932', '15986', '16097', '16098', '16161', '16199', '16203', '16211', '16472', '16491', '16501', '16533', '16581', '16625', '16627', '16633', '16634', '16652', '16653', '16661', '16692', '16708', '16748', '16762', '16772', '16856', '16864', '16873', '16874', '16878', '16879', '16914', '16985', '17009', '17014', '17026', '17168', '17208', '17279', '17281', '17301',

In [222]:
len(sp)

2694

In [223]:
qt

,5454,856,16121,9264,15627,10373,17843,10658,8203,9906,...,5216,16960,620,6690,9693,11332,14840,689,18090,15634
0,2.001019e-04,0.002313,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000019,0.000000,0.005494,0.000000,...,0.000000,0.000012,0.000281,0.000000,0.000000e+00,7.693530e-07,0.000000e+00,0.002389,0.000706,0.000047
1,2.922681e-04,0.002115,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000015,0.000000,0.005706,0.000000,...,0.000000,0.000006,0.000266,0.000000,0.000000e+00,6.496196e-07,0.000000e+00,0.001879,0.000628,0.000033
2,2.203969e-04,0.002277,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000021,0.000000,0.005890,0.000000,...,0.000000,0.000006,0.000277,0.000000,0.000000e+00,6.849161e-07,0.000000e+00,0.002695,0.000855,0.000076
3,0.000000e+00,0.002541,0.000000,0.000000e+00,0.000000e+00,5.460344e-07,0.000010,0.000000,0.004677,0.000003,...,0.000000,0.000004,0.000126,0.000000,0.000000e+00,1.973835e-06,1.674167e-06,0.003645,0.000880,0.000020
4,0.000000e+00,0.003120,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000004,0.000000,0.004996,0.000000,...,0.000000,0.000018,0.000210,0.000000,0.000000e+00,2.175747e-06,0.000000e+00,0.003862,0.000796,0.000036
12,6.934163e-05,0.003174,0.000000,0.000000e+00,4.886031e-07,0.000000e+00,0.000008,0.000000,0.004858,0.000000,...,0.000000,0.000006,0.000324,0.000000,0.000000e+00,2.029017e-06,0.000000e+00,0.003902,0.000884,0.000043
18,8.119938e-05,0.002884,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000009,0.000000,0.004565,0.000000,...,0.000000,0.000006,0.000300,0.000000,0.000000e+00,2.020758e-06,0.000000e+00,0.003151,0.001002,0.000023
19,1.303212e-04,0.003299,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000014,0.000000,0.005723,0.000000,...,0.000000,0.000013,0.000310,0.000000,0.000000e+00,6.177223e-07,0.000000e+00,0.002612,0.000751,0.000043
20,4.304759e-07,0.003257,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000012,0.000000,0.004674,0.000001,...,0.000000,0.000004,0.000243,0.000000,0.000000e+00,2.024950e-06,5.197454e-07,0.003162,0.001067,0.000031
21,4.110797e-05,0.003386,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000010,0.000000,0.005898,0.000000,...,0.000000,0.000008,0.000246,0.000000,0.000000e+00,5.749937e-07,0.000000e+00,0.002753,0.000904,0.000019


# Simplify and shorten the data for trial runs

In [ ]:
subset_spectra = spectra[0:200]
feature_ids = [str(spectrum.get("feature_id")) for spectrum in subset_spectra] # these are strings in matchms, assumed strings throughout
# feature_ids
subset_qt = quantification_table[ ["sample_id"] + feature_ids ]

In [ ]:
tmp_quantification_table = quantification_table.set_index("sample_id")
tmp_treatment_table = treatment_table.set_index("sample_id")
# Pandas pipe to align sample_id from quantification table and treatment table
(
  tmp_quantification_table.
  join(tmp_treatment_table, on="sample_id", how="left").
  reset_index()
  [["sample_id", "treatment"]]
)

In [ ]:
test_spectra = copy.deepcopy(subset_spectra)
test_quantification_table = copy.deepcopy(subset_qt)
test_treatment_data = copy.deepcopy(treatment_table)

import os
test_quantification_table.to_csv(os.path.join("test_data_medium","test_quant_table.csv"))
test_treatment_data.to_csv(os.path.join("test_data_medium", "test_treat_table.csv"))
matchms.exporting.save_as_mgf(test_spectra, os.path.join("test_data_medium","test_spectra.mgf"))